# Pre calculate Lambda solutions

Given a $H, E[\beta]$ input, we find the three-dimensional lambda solution.

Find k by k solutions, then find a smart, fast way to do the lookup
The lookup should find the row and column index where the solution lies
and also the four quadrants.

These solution take src.betas_transition as given, so can't be used for
different values of these

In [106]:
import numpy as np
from numba import njit
from scipy.stats import entropy
from scipy import optimize
from scipy.spatial.distance import euclidean

#Constants
betas_transition = np.array([-4., -2., -1.1])


def reparam_lambdas(x):
    """
    uses softmax to get values between 0 and 1
    and make them sum to 1
    """
    #return np.exp(x) / np.sum(np.exp(x))
    #Numerically more stable version
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


#@njit()
def h_and_exp_betas_eqns(orig_lambdas, βs, Eβ, H, w=np.array([[1., 0.], [0., 1./4.]])):
    """
    orig_lambdas: original lambda tries (not summing to zero, not within [0, 1])
    Eβ, H: the objectives
    βs: fixed constant of the model
    """
    lambdas = reparam_lambdas(orig_lambdas)
    g = np.array([entropy(lambdas) - H, np.dot(βs, lambdas) - Eβ])
    return g.T @ w @ g


def H_and_eb_to_lambda0(H, Eβ,
                        starting_values=np.array([0.1, 1.5, 1.])):
    """
    Generates a lambda0 vector from the values of
    the entropy and expected value of betas (H, EB)
    """

    def fun_(lambda_try):
        return h_and_exp_betas_eqns(lambda_try, betas_transition, Eβ, H)

    sol = optimize.minimize(fun_, x0=starting_values, method='Powell')
    lambdas_sol = reparam_lambdas(sol.x)
    if not sol.success:
        # Use Nelder-Mead from different starting_value
        sol = optimize.minimize(fun_, x0=np.array([1.6, 0.1, 0.25]),
                                method='Nelder-Mead')
        lambdas_sol = reparam_lambdas(sol.x)
        if not sol.success:
            print(f"Theta to lambda0 didn't converge", sol.x, lambdas_sol)

    return lambdas_sol



In [107]:
# Get candidates rows and cols
h_bounds = [0, np.log(3) + 0.05]
eb_bounds = [betas_transition[0], betas_transition[-1]]

h_n_digits_precision = 3
eb_n_digits_precision = 3
h_digit_precision = 0.01
eb_digit_precision = 0.01
h_candidates = np.arange(h_bounds[0], h_bounds[1], h_digit_precision)
eb_candidates = np.arange(eb_bounds[0], eb_bounds[1], eb_digit_precision)
grid_size = (len(h_candidates), len(eb_candidates))
grid_total_elements = grid_size[0]*grid_size[1]
lambda_values = np.empty((grid_size[0], grid_size[1], 3), dtype=np.float64)
print(grid_size)


(115, 290)


In [108]:
# Find values for candidates
import time
start = time.time()

for i_h, h in enumerate(h_candidates):
    for i_eb, eb in enumerate(eb_candidates):
        lambda_values[i_h, i_eb, :] = H_and_eb_to_lambda0(h, eb)
        
secs_it_took = time.time() - start
print(f"{secs_it_took / 60} minutos para array de tamaño {grid_size} o {grid_size[0]* grid_size[1]} elementos")

secs_por_element =  secs_it_took / grid_total_elements 
print(f"tomó {secs_por_element} minutos por valor")

5.142026197910309 minutos para array de tamaño (115, 290) o 33350 elementos


In [109]:
# Dictionary for quick lookup

h_dict = {}
for i in range(len(h_candidates)):
    h_dict[np.round(h_candidates[i], h_n_digits_precision)] = i

e_dict = {}
for i in range(len(eb_candidates)):
    e_dict[np.round(eb_candidates[i], eb_n_digits_precision)] = i
  

In [6]:
h_dict.keys()

dict_keys([0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1, 1.11, 1.12, 1.13, 1.14])

### OLS coeffs

In [194]:
import pandas as pd
import statsmodels.formula.api as smf
y1 = []
y2 = []
y3 = []
h_x = []
eb_x = []
for i_h, h in enumerate(h_candidates):
    for i_eb, eb in enumerate(eb_candidates):
        y1_, y2_, y3_ = lambda_values[i_h, i_eb, :]
        y1.append(y1_)
        y2.append(y2_)
        y3.append(y3_)
        h_x.append(h)
        eb_x.append(eb)
        
df = pd.DataFrame({'y1': y1, 'y2': y2, 'y3': y3,
                  'h_x': h_x, 'eb_x': eb_x})

df['h_x2'] = df.h_x**2
df['eb_x2'] = df.eb_x**2

formula1 = 'y1 ~ h_x * eb_x + h_x2*eb_x2 +  h_x2:eb_x - 1'
modely1 = smf.ols(formula=formula1, data=df).fit()

formula2 = 'y2 ~ h_x * eb_x + h_x2*eb_x2 + h_x2:eb_x- 1'
modely2 = smf.ols(formula=formula2, data=df).fit()

formula3 = 'y3 ~ h_x * eb_x + h_x2*eb_x2 + h_x2:eb_x - 1'
modely3 = smf.ols(formula=formula3, data=df).fit()

lambda1coeffs = modely1.params.values
lambda2coeffs = modely2.params.values
lambda3coeffs = modely3.params.values

@njit(['float64(float64, float64, float64, float64, float64[:])'])
def lambda_gen(h, eb, h2, eb2, lambdacoeffs):
    return (lambdacoeffs[0]*h + lambdacoeffs[1]*eb + 
           lambdacoeffs[2]*h*eb + lambdacoeffs[3]*h2
           + lambdacoeffs[4]*eb2 + lambdacoeffs[5]*(h2*eb2)
           + lambdacoeffs[6]*(h2*eb))

@njit(['float64[:](float64, float64, float64[:], float64[:], float64[:])'])
def all_lambda_gen(h, eb, lcoeffs1, lcoeffs2, lcoeffs3):
    h2 = h**2
    eb2 = eb**2
    return np.array([lambda_gen(h, eb, h2, eb2, lcoeffs1),
                    lambda_gen(h, eb, h2, eb2, lcoeffs2),
                    lambda_gen(h, eb, h2, eb2, lcoeffs3)])


h = 0.2
eb = -2.
print(lambda_gen(h, eb, h**2, eb**2, lambda1coeffs))
print(lambda_gen(h, eb, h**2, eb**2, lambda2coeffs))
print(lambda_gen(h, eb, h**2, eb**2, lambda3coeffs))

-0.002408099160052836
0.9365117449459819
0.07218528485557224


In [195]:
all_lambda_gen(h, eb, lambda1coeffs, lambda2coeffs, lambda3coeffs)

array([-0.0024081 ,  0.93651174,  0.07218528])

## Save lambda_matrix_dict

In [191]:
import pickle
lambda_matrix_dict = {}
lambda_matrix_dict['lambda_matrix'] = lambda_values
lambda_matrix_dict['h_candidates'] = h_candidates
lambda_matrix_dict['eb_candidates'] = eb_candidates
lambda_matrix_dict['h_n_digits_precision'] = h_n_digits_precision
lambda_matrix_dict['eb_n_digits_precision'] = eb_n_digits_precision
lambda_matrix_dict['h_dict'] = h_dict
lambda_matrix_dict['e_dict'] = e_dict
    lambda_matrix_dict['lambda1coeffs'] = lambda1coeffs
    lambda_matrix_dict['lambda2coeffs'] = lambda2coeffs
    lambda_matrix_dict['lambda3coeffs'] = lambda3coeffs

with open('../../data/lambda_matrix_dict.pickle', 'wb') as f:
    pickle.dump(lambda_matrix_dict, f)



## Speeding up stuff

### Linear interpolation

Think of each $\lambda$ as the $y$s and $H, eb$ as $x_1, x_2$ 

for $y_1, y_2, y_3 $:

$$ y_1 = \beta_0 + \beta_1 x_1 + \beta_2 x_1^2 + \beta_3 x_2 + \beta_4 x_2^2 + \beta_5 x_1x_2 + \beta_6 x_1^2 x_2 + \beta_7 x_1 x_2^2 $$

Store those $7*3 = 21$ coefficients and your $f$ will be just this.
It's important to verify that this linear regression function has relatively good $R^2$

In [119]:
lambda_values.shape

(115, 290, 3)

-0.002408099160052836
0.9365117449459819
0.07218528485557224


In [114]:
%%timeit
input_heb_to_lambda_con_norm(input_point, lambda_values, h_candidates, eb_candidates,
                                 h_n_digits_precision, eb_n_digits_precision,
                                 h_dict, e_dict)

78.7 µs ± 841 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [116]:
nb_input_heb_to_lambda_con_norm(input_point, lambda_values, h_candidates, eb_candidates,
                                 h_n_digits_precision, eb_n_digits_precision,
                                 h_dict, e_dict)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Internal error at <numba.typeinfer.ArgConstraint object at 0x11fd6b860>:
--%<----------------------------------------------------------------------------
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/spike_basicoV3/lib/python3.6/site-packages/numba/errors.py", line 609, in new_error_context
    yield
  File "/usr/local/anaconda3/envs/spike_basicoV3/lib/python3.6/site-packages/numba/typeinfer.py", line 198, in __call__
    assert ty.is_precise()
AssertionError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/spike_basicoV3/lib/python3.6/site-packages/numba/typeinfer.py", line 141, in propagate
    constraint(typeinfer)
  File "/usr/local/anaconda3/envs/spike_basicoV3/lib/python3.6/site-packages/numba/typeinfer.py", line 199, in __call__
    typeinfer.add_type(self.dst, ty, loc=self.loc)
  File "/usr/local

In [ ]:
%%timeit
input_point = np.array([0.2134, -3.977])
#Find row, col
H, eb = input_point[0], input_point[1]
row = h_dict[np.round(H, h_n_digits_precision-1)]
col = e_dict[np.round(eb, eb_n_digits_precision-1)]
distances = numba_from_input_heb_to_lambda(input_point, row, col, lambda_values)

In [ ]:
H, eb = input_point[0], input_point[1]
row = h_dict[np.round(H, h_n_digits_precision-1)]
col = e_dict[np.round(eb, eb_n_digits_precision-1)]
stacked_values = numba_from_input_heb_to_lambda(input_point, row, col, lambda_values)


In [ ]:

def from_input_heb_to_lambda(input_point, lambda_values, h_candidates, eb_candidates,
                                h_n_digits_precision, eb_n_digits_precision):
    H, eb = input_point[0], input_point[1]
    #Find row, col
    row = h_dict[np.round(H, h_n_digits_precision-1)]
    col = e_dict[np.round(eb, eb_n_digits_precision-1)]

    # Distances to row-1, row, row+1 and col-1, col, col+1
    dist_row = np.array([np.abs(H - h_candidates[row - 1]), np.abs(H - h_candidates[row]), 
               np.abs(H - h_candidates[row + 1])])
    if np.argmax(dist_row) == 2:
        relevant_rows = [row-1, row]
        dist_row = dist_row[0:2]
    elif np.argmax(dist_row) == 0:
        relevant_rows = [row, row+1]
        dist_row = dist_row[1::]

    dist_col = np.array([np.abs(eb- eb_candidates[col - 1]), np.abs(eb - eb_candidates[col]), 
               np.abs(eb - eb_candidates[col + 1])])
    if np.argmax(dist_col) == 2:
        relevant_cols = [col-1, col]
        dist_col = dist_col[0:2]
    elif np.argmax(dist_col) == 0:
        relevant_cols = [col, col+1]
        dist_col = dist_col[1::]

    pointA, pointB = [relevant_rows[0], relevant_cols[0]], [relevant_rows[0], relevant_cols[1]]
    pointC, pointD = [relevant_rows[1], relevant_cols[0]], [relevant_rows[1], relevant_cols[1]]
    points = [pointA, pointB, pointC, pointD]

    pointA_value = lambda_values[relevant_rows[0], relevant_cols[0]]
    pointB_value = lambda_values[relevant_rows[0], relevant_cols[1]]
    pointC_value = lambda_values[relevant_rows[1], relevant_cols[0]]
    pointD_value = lambda_values[relevant_rows[1], relevant_cols[1]]
    values = np.array([pointA_value, pointB_value, pointC_value, pointD_value])
    distances = np.array([np.linalg.norm(input_point - point) for point in points])
    distances /= distances.sum()
    
    #Linear combination of A, B, C, D points
    return distances[np.newaxis, :]@values

from numba import njit



@njit()
def numba_from_input_heb_to_lambda(input_point, row, col, lambda_values):
    H, eb = input_point[0], input_point[1]

    # Distances to row-1, row, row+1 and col-1, col, col+1
    dist_row = np.array([np.abs(H - h_candidates[row - 1]), np.abs(H - h_candidates[row]), 
               np.abs(H - h_candidates[row + 1])])
    if np.argmax(dist_row) == 2:
        relevant_rows = [row-1, row]
        dist_row = dist_row[0:2]
    elif np.argmax(dist_row) == 0:
        relevant_rows = [row, row+1]
        dist_row = dist_row[1::]

    dist_col = np.array([np.abs(eb- eb_candidates[col - 1]), np.abs(eb - eb_candidates[col]), 
               np.abs(eb - eb_candidates[col + 1])])
    if np.argmax(dist_col) == 2:
        relevant_cols = np.array([col-1, col])
        dist_col = dist_col[0:2]
    elif np.argmax(dist_col) == 0:
        relevant_cols = np.array([col, col+1])
        dist_col = dist_col[1::]

    points = np.array([[relevant_rows[0], relevant_cols[0]], [relevant_rows[0], relevant_cols[1]],
                       [relevant_rows[1], relevant_cols[0]], [relevant_rows[1], relevant_cols[1]]])

    distances = np.array([np.linalg.norm(input_point - points[0]), np.linalg.norm(input_point - points[1]),
                         np.linalg.norm(input_point - points[2]), np.linalg.norm(input_point - points[3])])
    distances /= distances.sum()
    
    #Linear combination of A, B, C, D points
    #return (distances.reshape((-1, 1)) @\
    #        np.array([lambda_values[relevant_rows[0], relevant_cols[0]], lambda_values[relevant_rows[0], relevant_cols[1]],
    #                     lambda_values[relevant_rows[1], relevant_cols[0]], lambda_values[relevant_rows[1], relevant_cols[1]]]))
    #return distances.reshape((-1, 1))
    #return np.array([lambda_values[relevant_rows[0], relevant_cols[0]], lambda_values[relevant_rows[0], relevant_cols[1]],
     #                    lambda_values[relevant_rows[1], relevant_cols[0]], lambda_values[relevant_rows[1], relevant_cols[1]]])
    #return relevant_rows, relevant_cols
    stacked_values = np.vstack((lambda_values[relevant_rows[0], relevant_cols[0]], lambda_values[relevant_rows[0], relevant_cols[1]],
                         lambda_values[relevant_rows[1], relevant_cols[0]], lambda_values[relevant_rows[1], relevant_cols[1]]))

    return stacked_values
    #return distances.reshape((-1, 1)) @ stacked_values